In [1]:
!pip install pyspark py4js

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 5.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=2d64b84a0dd57a52c39db4b41539db6162fb47c29812c25710144b054c9771eb
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
  Created wheel for py4js: filename=py4js-0.1.2-py3-none-any.whl size=5573 sha256=a59756b7c16dbbec428a0e9b2f813e2aca7ba072fc78c8cf9ec9776d1a9d434d
  Stored in directory: /root/.cache/pip/wheels/9d/ac/0b/a12e14a0341c39773f23df70bcecdeeacdc553430829e2d1a4
Successfully built pyspark py4js


In [5]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName("pyspark_practical_session_3").getOrCreate()

In [9]:
df_inferschema = spark.read.format("csv")\
.option("header","true")\
.option("inferscehma","true")\
.load("/content/drive/MyDrive/Colab Notebooks/manish_kumar_practicals/data_corrupted_records.csv")
df_inferschema.show()

+---+--------+---+------+------------+--------+
| id|    name|age|salary|     address| nominee|
+---+--------+---+------+------------+--------+
|  1|  Manish| 26| 75000|       bihar|nominee1|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2|
|  3|  Pritam| 22|150000|   Bangalore|   India|
|  4|Prantosh| 17|200000|     Kolkata|   India|
|  5|  Vikash| 31|300000|        NULL|nominee5|
+---+--------+---+------+------------+--------+



In [7]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [12]:
schema_df = StructType([
    StructField("id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("age",IntegerType(),False),
    StructField("salary",IntegerType(),False),
    StructField("address", StringType(), False),
    StructField("nominee", StringType(), False)
])

df_with_schema = spark.read.format("csv")\
.option("mode","DROPMALFORMED")\
.option("badRecordsPath","/content/drive/MyDrive/Colab Notebooks/manish_kumar_practicals")\
.option("header","true")\
.schema(schema_df)\
.load("/content/drive/MyDrive/Colab Notebooks/manish_kumar_practicals/data_corrupted_records.csv")

df_with_schema.show()

+---+------+---+------+------------+--------+
| id|  name|age|salary|     address| nominee|
+---+------+---+------+------------+--------+
|  1|Manish| 26| 75000|       bihar|nominee1|
|  2|Nikita| 23|100000|uttarpradesh|nominee2|
|  5|Vikash| 31|300000|        NULL|nominee5|
+---+------+---+------+------------+--------+



In [13]:
df_with_schema.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- address: string (nullable = true)
 |-- nominee: string (nullable = true)



In [14]:
df_with_schema.columns

['id', 'name', 'age', 'salary', 'address', 'nominee']

In [18]:
#simply access column by passing column name
df_with_schema.select("name").show()

+--------+
|    name|
+--------+
|  Manish|
|  Nikita|
|  Pritam|
|Prantosh|
|  Vikash|
+--------+



In [16]:
from pyspark.sql.functions import *

In [19]:
#access the column by using col and passing column name inside
df_with_schema.select(col("name")).show()

+--------+
|    name|
+--------+
|  Manish|
|  Nikita|
|  Pritam|
|Prantosh|
|  Vikash|
+--------+



In [20]:
#issue comes with passing normal name
df_with_schema.select("id +5").show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `id +5` cannot be resolved. Did you mean one of the following? [`id`, `age`, `name`, `address`, `salary`].;
'Project ['id +5]
+- Relation [id#224,name#225,age#226,salary#227,address#228,nominee#229] csv


In [21]:
#above error we face as this column is id and "+ 5" is not a column name

In [24]:
#to avoid above issue we can by pass this with help of col
df_with_schema.select(col("id")+5).show()

+--------+
|(id + 5)|
+--------+
|       6|
|       7|
|       8|
|       9|
|      10|
+--------+



In [25]:
#select multiple columns
df_with_schema.select("id","name","age").show()

+---+--------+---+
| id|    name|age|
+---+--------+---+
|  1|  Manish| 26|
|  2|  Nikita| 23|
|  3|  Pritam| 22|
|  4|Prantosh| 17|
|  5|  Vikash| 31|
+---+--------+---+



In [27]:
#select multiple columns through cols,
#but avoid using this when we just want to access columns, use single string approach only
df_with_schema.select(col("id"),col("name"),col("age")).show()

+---+--------+---+
| id|    name|age|
+---+--------+---+
|  1|  Manish| 26|
|  2|  Nikita| 23|
|  3|  Pritam| 22|
|  4|Prantosh| 17|
|  5|  Vikash| 31|
+---+--------+---+



In [29]:
#Multiple approaches all at once
#string approach
#col approach
#passing col name in [""] like pandas
#retreiving column like in sql i.e: .column_name

df_with_schema.select(
    "id",
    col("name"),
    df_with_schema["age"],
    df_with_schema.salary
).show()


+---+--------+---+------+
| id|    name|age|salary|
+---+--------+---+------+
|  1|  Manish| 26| 75000|
|  2|  Nikita| 23|100000|
|  3|  Pritam| 22|150000|
|  4|Prantosh| 17|200000|
|  5|  Vikash| 31|300000|
+---+--------+---+------+



In [30]:
# df_name.column_name used in joins specially

In [32]:
#Solution for below issue

#issue comes with passing normal name
#code--> df_with_schema.select("id +5").show()

#solution will be with help of expressions

df_with_schema.select(expr("id +5")).show()

+--------+
|(id + 5)|
+--------+
|       6|
|       7|
|       8|
|       9|
|      10|
+--------+



In [33]:
#helps in aliasing

df_with_schema.select(expr("id as employee_id"), expr("name as employee_name"), expr("concat(name,address)")).show()

+-----------+-------------+---------------------+
|employee_id|employee_name|concat(name, address)|
+-----------+-------------+---------------------+
|          1|       Manish|          Manishbihar|
|          2|       Nikita|   Nikitauttarpradesh|
|          3|       Pritam|      PritamBangalore|
|          4|     Prantosh|      PrantoshKolkata|
|          5|       Vikash|                 NULL|
+-----------+-------------+---------------------+



In [35]:
#how to write same code in spark sql

#convert df to temporary view table

In [36]:
df_with_schema.createOrReplaceTempView("df_with_schema_tbl")

In [38]:
spark.sql("""

select * from df_with_schema_tbl

""").show()

+---+------+---+------+------------+--------+
| id|  name|age|salary|     address| nominee|
+---+------+---+------+------------+--------+
|  1|Manish| 26| 75000|       bihar|nominee1|
|  2|Nikita| 23|100000|uttarpradesh|nominee2|
|  5|Vikash| 31|300000|        NULL|nominee5|
+---+------+---+------+------------+--------+



In [40]:
spark.sql("""

select id, name, salary from df_with_schema_tbl

""").show()

+---+--------+------+
| id|    name|salary|
+---+--------+------+
|  1|  Manish| 75000|
|  2|  Nikita|100000|
|  3|  Pritam|150000|
|  4|Prantosh|200000|
|  5|  Vikash|300000|
+---+--------+------+



In [39]:
#similarly in dataframe also we can select all columns
df_with_schema.select("*").show()

+---+------+---+------+------------+--------+
| id|  name|age|salary|     address| nominee|
+---+------+---+------+------------+--------+
|  1|Manish| 26| 75000|       bihar|nominee1|
|  2|Nikita| 23|100000|uttarpradesh|nominee2|
|  5|Vikash| 31|300000|        NULL|nominee5|
+---+------+---+------+------------+--------+

